In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 14

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
# aisles = pd.read_csv('../../data/aisles.csv')
# departments = pd.read_csv('../../data/departments.csv')
order_products_prior = pd.read_csv('../../data/order_products__prior.csv')
order_products_train = pd.read_csv('../../data/order_products__train.csv')
orders = pd.read_csv('../../data/orders.csv')
# products = pd.read_csv('../../data/products.csv')
order_test = pd.read_csv('../../data/sample_submission.csv')

# order_products_prior_df = pd.read_csv('../../data/order_products_prior_df.csv')
# prior_orders_with_product = pd.read_csv('../../data/prior_orders_with_product.csv') 

Creating base table by joining prior and training data. For the purpose of this project I'm removing the test dataset, and instead portioning out some of the training set to use as test data, to be able to measure accuracy. 

In [4]:
train = order_products_train
prior = order_products_prior

train = train.merge(orders, on='order_id', how='left')
prior = prior.merge(orders, on='order_id', how='left')

new = train.append(prior, ignore_index=True)
new['target'] = 0
new.loc[new['eval_set'] == 'train', 'target'] = 1
new.loc[new['eval_set'] == 'test', 'target'] = 2

In [5]:
user_products = new[(new.target==0)|(new.target==1)]
user_products_test = new[new.eval_set=='test']

In [6]:
# # order_id, product_id, add_to_cart_order, reorered, product_name, aisle_id, department_id, aisle, department 
# order_products_prior_df.head()
# order_products_prior_df.drop('Unnamed: 0',axis=1,inplace=True)

# #order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order, num_orders, num_products
# prior_orders_with_product.head()
# prior_orders_with_product.drop('Unnamed: 0',axis=1,inplace=True)

# user_products = new
user_products


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,target
0,1,49302,1,1,112108,train,4,4,10,9.000,1
1,1,11109,2,1,112108,train,4,4,10,9.000,1
2,1,10246,3,0,112108,train,4,4,10,9.000,1
3,1,49683,4,0,112108,train,4,4,10,9.000,1
4,1,43633,5,1,112108,train,4,4,10,9.000,1
...,...,...,...,...,...,...,...,...,...,...,...
33819101,3421083,39678,6,1,25247,prior,24,2,6,21.000,0
33819102,3421083,11352,7,0,25247,prior,24,2,6,21.000,0
33819103,3421083,4600,8,0,25247,prior,24,2,6,21.000,0
33819104,3421083,24852,9,1,25247,prior,24,2,6,21.000,0


In [7]:
# removing net new products in training set
user_products = user_products[~((user_products['eval_set']=='train') & (user_products['reordered']==0))]

In [8]:
user_products

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,target
0,1,49302,1,1,112108,train,4,4,10,9.000,1
1,1,11109,2,1,112108,train,4,4,10,9.000,1
4,1,43633,5,1,112108,train,4,4,10,9.000,1
7,1,22035,8,1,112108,train,4,4,10,9.000,1
9,36,19660,2,1,79431,train,23,6,18,30.000,1
...,...,...,...,...,...,...,...,...,...,...,...
33819101,3421083,39678,6,1,25247,prior,24,2,6,21.000,0
33819102,3421083,11352,7,0,25247,prior,24,2,6,21.000,0
33819103,3421083,4600,8,0,25247,prior,24,2,6,21.000,0
33819104,3421083,24852,9,1,25247,prior,24,2,6,21.000,0


In [9]:
unique_user_product = user_products[user_products['user_id'].isin(orders[orders['eval_set']=='train']['user_id'])].drop_duplicates(['user_id', 'product_id'])[['user_id', 'product_id','eval_set']]
unique_user_product[unique_user_product.duplicated(subset=['user_id','product_id'])==True]


,user_id,product_id,eval_set


In [10]:
unique_user_product['target'] = 0 
unique_user_product.loc[unique_user_product['eval_set'] == 'train', 'target'] = 1
unique_user_product.target.value_counts()

0    7645837
1     828824
Name: target, dtype: int64

In [11]:
unique_user_product = unique_user_product.merge(orders[orders['eval_set']=='train'][['order_id','user_id']], on='user_id', how='left')
unique_user_product
del orders
del train
del prior

## Product Stats

The purpose of this section is to pull order metrics aggregated by product. First we do a running count of products ordered group by the user and product. The max(row_num) will output how many times a user ordered a product across all orders. 

In [12]:
user_products['row_num'] = user_products.groupby(['user_id','product_id']).cumcount()+1

<ipython-input-12-63e2e57afd6b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_products['row_num'] = user_products.groupby(['user_id','product_id']).cumcount()+1


In [13]:
user_products.shape

(33263313, 12)

I'm curious to see reorder rates between first and second orders of a product. The underlying assumption here is that something that is ordered more than once indicates some interest in the product to be ordered again. The prod_first_orders and prod_second_orders dfs will output how many times a product was ordered once or twice by the same user respectively.

In [14]:
first_order = user_products[user_products.row_num ==1]
second_order = user_products[user_products.row_num ==2]

In [15]:
prod_orders = user_products.groupby(['product_id']).agg({'order_id':'count'})
prod_orders

,order_id
product_id,
1,1901
2,91
3,283
4,343
5,16
...,...
49684,9
49685,49
49686,125


In [16]:
prod_reorders = user_products.groupby(['product_id']).agg({'reordered':'sum'})
prod_reorders

,reordered
product_id,
1,1185
2,13
3,209
4,161
5,10
...,...
49684,1
49685,6
49686,89


In [17]:
prod_first_orders = first_order.groupby('product_id').agg({'row_num':'sum'})
prod_first_orders

,row_num
product_id,
1,716
2,78
3,74
4,182
5,6
...,...
49684,8
49685,43
49686,36


In [18]:
prod_second_orders = second_order.groupby('product_id').agg({'row_num':'sum'})
prod_second_orders

,row_num
product_id,
1,578
2,18
3,74
4,134
5,8
...,...
49684,2
49685,12
49686,34


In [19]:
prod_df = pd.merge(prod_orders, prod_reorders, how='left', on='product_id')
prod_df = pd.merge(prod_df, prod_first_orders, how='left', on='product_id')
prod_df = pd.merge(prod_df, prod_second_orders, how='left', on='product_id')

prod_df.rename(columns={'order_id':'ordered','row_num_x':'num_users_order_product_once','row_num_y':'num_users_order_product_twice'},inplace=True)

prod_df

,ordered,reordered,num_users_order_product_once,num_users_order_product_twice
product_id,,,,
1,1901,1185,716,578.000
2,91,13,78,18.000
3,283,209,74,74.000
4,343,161,182,134.000
5,16,10,6,8.000
...,...,...,...,...
49684,9,1,8,2.000
49685,49,6,43,12.000
49686,125,89,36,34.000


In [20]:
del prod_reorders
del prod_first_orders
del prod_second_orders

In [21]:
prod_df['reorder_ratio'] = prod_df.num_users_order_product_twice/prod_df.num_users_order_product_once
prod_df['times_reordered'] = (prod_df.reordered / prod_df.num_users_order_product_once)+1
prod_df['reorders_prob'] = prod_df.reordered / prod_df.ordered

prod_df

,ordered,reordered,num_users_order_product_once,num_users_order_product_twice,reorder_ratio,times_reordered,reorders_prob
product_id,,,,,,,
1,1901,1185,716,578.000,0.807,2.655,0.623
2,91,13,78,18.000,0.231,1.167,0.143
3,283,209,74,74.000,1.000,3.824,0.739
4,343,161,182,134.000,0.736,1.885,0.469
5,16,10,6,8.000,1.333,2.667,0.625
...,...,...,...,...,...,...,...
49684,9,1,8,2.000,0.250,1.125,0.111
49685,49,6,43,12.000,0.279,1.140,0.122
49686,125,89,36,34.000,0.944,3.472,0.712


In [22]:
# prod_df.drop(['reorders_to_orders','reorder_probability'],axis=1,inplace=True)
# prod_df.rename(columns={'reorder_ratio':'order_twice_ratio','reorders_prob':'overall_reorder_prob'})

### User-Product

In [23]:
user_product_df = user_products.groupby(['user_id','product_id']).agg(user_product_count=("order_number","count"),
                                                                          user_product_first_order=("order_number","min"),
                                                                          user_product_last_order=("order_number","max"),
                                                                          user_product_avg_basket_placement=("add_to_cart_order","mean"))



In [24]:
user_product_df = user_product_df.reset_index()

In [25]:
user_product_df[user_product_df.duplicated(subset=['user_id','product_id'])==True]
# temp = user_product_df.drop_duplicates(['user_id', 'product_id'])[['user_id', 'product_id','user_product_count','user_product_first_order','user_product_last_order','user_product_avg_basket_placement']]


,user_id,product_id,user_product_count,user_product_first_order,user_product_last_order,user_product_avg_basket_placement


## User Stats

### Product

In [26]:
print(user_products.user_id.nunique())
user_products

206209


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,target,row_num
0,1,49302,1,1,112108,train,4,4,10,9.000,1,1
1,1,11109,2,1,112108,train,4,4,10,9.000,1,1
4,1,43633,5,1,112108,train,4,4,10,9.000,1,1
7,1,22035,8,1,112108,train,4,4,10,9.000,1,1
9,36,19660,2,1,79431,train,23,6,18,30.000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
33819101,3421083,39678,6,1,25247,prior,24,2,6,21.000,0,3
33819102,3421083,11352,7,0,25247,prior,24,2,6,21.000,0,1
33819103,3421083,4600,8,0,25247,prior,24,2,6,21.000,0,1
33819104,3421083,24852,9,1,25247,prior,24,2,6,21.000,0,13


In [27]:
prod_counts= user_products.groupby('user_id').agg(total_products=('product_id','count'),
                                                  reordered_count=('reordered','sum'))

In [28]:
prod_unique = user_products.groupby('user_id').product_id.nunique().reset_index()
prod_unique.columns = ['user_id', 'unique_product_count']

user_prod_counts = pd.merge(prod_counts, prod_unique, how='left',on='user_id')

In [29]:
reordered = user_products[user_products.reordered ==1]
reordered_unique = reordered.groupby('user_id').product_id.nunique().reset_index()
reordered_unique.columns=['user_id','unique_reordered_count']

user_prod_counts= pd.merge(user_prod_counts, reordered_unique, how='left',on='user_id')

Getting count of total products purchased over customer lifetime vs unique products \
Also number of total reordered products ever purchased, and number of unique reordered products

In [30]:
user_prod_counts.head()
user_prod_counts['prob_unique'] = user_prod_counts.unique_product_count/user_prod_counts.total_products
user_prod_counts['prob_reordered_unique'] = user_prod_counts.unique_reordered_count/user_prod_counts.reordered_count
user_prod_counts.head()

,user_id,total_products,reordered_count,unique_product_count,unique_reordered_count,prob_unique,prob_reordered_unique
0,1,69,51,18,12.000,0.261,0.235
1,2,207,105,102,42.000,0.493,0.400
2,3,88,55,33,19.000,0.375,0.345
3,4,18,1,17,1.000,0.944,1.000
4,5,41,18,23,10.000,0.561,0.556


Calculating the reorder ratio per user\
reorder count is how many times user reordered any product \
multi_order_count is count of all products where there are multiple orders (total products -first order)

In [31]:
multi_order_user = user_products[user_products.order_number > 1]
sum_multi_orders = multi_order_user.groupby('user_id').agg(multi_order_count=('order_number','count'))
sum_reordered = reordered.groupby('user_id').agg(reordered_count=('reordered','sum'))

user_reorder_ratio = pd.merge(sum_reordered, sum_multi_orders,how='left',on='user_id')

In [32]:
user_reorder_ratio['reorder_ratio'] = user_reorder_ratio.reordered_count/user_reorder_ratio.multi_order_count
user_reorder_ratio

,reordered_count,multi_order_count,reorder_ratio
user_id,,,
1,51,64,0.797
2,105,194,0.541
3,55,78,0.705
4,1,14,0.071
5,18,30,0.600
...,...,...,...
206205,14,21,0.667
206206,135,281,0.480
206207,131,199,0.658


In [33]:
user_stats = pd.merge(user_prod_counts, user_reorder_ratio, how='left',on='user_id')
user_stats.drop('reordered_count_y',axis=1, inplace=True)
user_stats.rename(columns={'reordered_count_x':'total_products_reordered','prob_unique':'perc_unique','prob_reordered_unique':'perc_reordered_unique'},inplace=True)

### Orders

In [34]:
total_orders = user_products.groupby('user_id').agg(total_orders=('order_number','max'))

In [35]:
user_stats = pd.merge(user_stats,total_orders,how='left',on='user_id')
user_stats

,user_id,total_products,total_products_reordered,unique_product_count,unique_reordered_count,perc_unique,perc_reordered_unique,multi_order_count,reorder_ratio,total_orders
0,1,69,51,18,12.000,0.261,0.235,64.000,0.797,11
1,2,207,105,102,42.000,0.493,0.400,194.000,0.541,15
2,3,88,55,33,19.000,0.375,0.345,78.000,0.705,12
3,4,18,1,17,1.000,0.944,1.000,14.000,0.071,5
4,5,41,18,23,10.000,0.561,0.556,30.000,0.600,5
...,...,...,...,...,...,...,...,...,...,...
206204,206205,38,14,24,10.000,0.632,0.714,21.000,0.667,4
206205,206206,285,135,150,44.000,0.526,0.326,281.000,0.480,67
206206,206207,223,131,92,48.000,0.413,0.366,199.000,0.658,16
206207,206208,677,479,198,95.000,0.292,0.198,665.000,0.720,49


In [36]:
user_time_dim = user_products.groupby('user_id').agg(customer_lifetime_days = ('days_since_prior_order','sum'),
                                     avg_days_between_orders= ('days_since_prior_order','mean'),
                                     max_time_between_orders= ('days_since_prior_order', 'max'),
                                     min_time_between_orders= ('days_since_prior_order','min'))


In [37]:
user_stats = pd.merge(user_stats, user_time_dim,how='left',on='user_id')
user_stats


,user_id,total_products,total_products_reordered,unique_product_count,unique_reordered_count,perc_unique,perc_reordered_unique,multi_order_count,reorder_ratio,total_orders,customer_lifetime_days,avg_days_between_orders,max_time_between_orders,min_time_between_orders
0,1,69,51,18,12.000,0.261,0.235,64.000,0.797,11,1234.000,19.281,30.000,0.000
1,2,207,105,102,42.000,0.493,0.400,194.000,0.541,15,3266.000,16.835,30.000,3.000
2,3,88,55,33,19.000,0.375,0.345,78.000,0.705,12,896.000,11.487,21.000,7.000
3,4,18,1,17,1.000,0.944,1.000,14.000,0.071,5,215.000,15.357,21.000,0.000
4,5,41,18,23,10.000,0.561,0.556,30.000,0.600,5,401.000,13.367,19.000,6.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206204,206205,38,14,24,10.000,0.632,0.714,21.000,0.667,4,370.000,17.619,30.000,10.000
206205,206206,285,135,150,44.000,0.526,0.326,281.000,0.480,67,1136.000,4.043,15.000,0.000
206206,206207,223,131,92,48.000,0.413,0.366,199.000,0.658,16,2961.000,14.879,30.000,1.000
206207,206208,677,479,198,95.000,0.292,0.198,665.000,0.720,49,4949.000,7.442,20.000,0.000


In [38]:
user_stats['avg_cart_size'] = user_stats.total_products/user_stats.total_orders

user_stats

,user_id,total_products,total_products_reordered,unique_product_count,unique_reordered_count,perc_unique,perc_reordered_unique,multi_order_count,reorder_ratio,total_orders,customer_lifetime_days,avg_days_between_orders,max_time_between_orders,min_time_between_orders,avg_cart_size
0,1,69,51,18,12.000,0.261,0.235,64.000,0.797,11,1234.000,19.281,30.000,0.000,6.273
1,2,207,105,102,42.000,0.493,0.400,194.000,0.541,15,3266.000,16.835,30.000,3.000,13.800
2,3,88,55,33,19.000,0.375,0.345,78.000,0.705,12,896.000,11.487,21.000,7.000,7.333
3,4,18,1,17,1.000,0.944,1.000,14.000,0.071,5,215.000,15.357,21.000,0.000,3.600
4,5,41,18,23,10.000,0.561,0.556,30.000,0.600,5,401.000,13.367,19.000,6.000,8.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206204,206205,38,14,24,10.000,0.632,0.714,21.000,0.667,4,370.000,17.619,30.000,10.000,9.500
206205,206206,285,135,150,44.000,0.526,0.326,281.000,0.480,67,1136.000,4.043,15.000,0.000,4.254
206206,206207,223,131,92,48.000,0.413,0.366,199.000,0.658,16,2961.000,14.879,30.000,1.000,13.938
206207,206208,677,479,198,95.000,0.292,0.198,665.000,0.720,49,4949.000,7.442,20.000,0.000,13.816


In [39]:
# orders.drop('num_orders',axis=1,inplace=True)
orders_train = user_products[user_products.eval_set == 'train']
orders_train

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,target,row_num
0,1,49302,1,1,112108,train,4,4,10,9.000,1,1
1,1,11109,2,1,112108,train,4,4,10,9.000,1,1
4,1,43633,5,1,112108,train,4,4,10,9.000,1,1
7,1,22035,8,1,112108,train,4,4,10,9.000,1,1
9,36,19660,2,1,79431,train,23,6,18,30.000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1384612,3421063,14233,3,1,169679,train,30,0,10,4.000,1,1
1384613,3421063,35548,4,1,169679,train,30,0,10,4.000,1,1
1384614,3421070,35951,1,1,139822,train,15,6,10,8.000,1,1
1384615,3421070,16953,2,1,139822,train,15,6,10,8.000,1,1


In [40]:
# prod_df=prod_df.reset_index()
prod_df
prod_df.rename(columns={'ordered':'product_ordered_vol', 
                        'reordered':'product_reordered_vol',
                        'num_users_order_product_once': 'product_ordered_once_vol',
                        'num_users_order_product_twice': 'product_ordered_twice_vol',
                        'reorder_ratio':'product_order_twice_ratio',
                        'times_reordered':'product_avg_reorders',
                        'reorders_prob':'product_overall_reorder_prob'},inplace=True)

In [41]:
user_stats
user_stats.rename(columns={'total_products':'user_total_products',
                           'total_products_reordered':'user_total_products_reordered',
                           'unique_product_count':'user_unique_product_count',
                           'unique_reordered_count':'user_unique_reorder_count',
                           'perc_unique':'user_unique_product_perc',
                           'perc_reordered_unique': 'user_unique_reorder_perc',
                           'multi_order_count':'user_total_items_after_first_order',
                           'reorder_ratio':'user_reorder_ratio',
                           'total_orders':'user_total_orders',
                           'customer_lifetime_days':'user_lifetime_days',
                           'avg_days_between_orders':'user_avg_days_between_orders',
                           'max_time_between_orders':'user_max_time_between_orders',
                           'min_time_between_orders':'user_min_time_between_orders',
                           'avg_cart_size':'user_avg_cart_size'},inplace=True)
                           

In [42]:
# user_product_df= user_product_df.reset_index()
user_product_full = user_product_df.merge(prod_df, how='left',on='product_id').merge(user_stats, how='left', on='user_id')

In [43]:
user_product_full.shape

(13307953, 27)

In [44]:
del prod_df
del user_stats

In [45]:
user_product_full['user_product_order_rate'] = user_product_full.user_product_count/user_product_full.user_total_orders
user_product_full['user_product_reorder_rate'] = (user_product_full.user_product_count / 
                                                  (user_product_full.user_total_orders-
                                                   user_product_full.user_product_first_order +1))
user_product_full['user_product_last_time_product_ordered'] = user_product_full.user_total_orders - user_product_full.user_product_last_order


In [46]:
user_product_full

,user_id,product_id,user_product_count,user_product_first_order,user_product_last_order,user_product_avg_basket_placement,product_ordered_vol,product_reordered_vol,product_ordered_once_vol,product_ordered_twice_vol,...,user_reorder_ratio,user_total_orders,user_lifetime_days,user_avg_days_between_orders,user_max_time_between_orders,user_min_time_between_orders,user_avg_cart_size,user_product_order_rate,user_product_reorder_rate,user_product_last_time_product_ordered
0,1,196,11,1,11,1.364,37012,29012,8000,9614.000,...,0.797,11,1234.000,19.281,30.000,0.000,6.273,1.000,1.000,0
1,1,10258,10,2,11,3.600,2024,1467,557,660.000,...,0.797,11,1234.000,19.281,30.000,0.000,6.273,0.909,1.000,0
2,1,10326,1,5,5,5.000,5809,3886,1923,2178.000,...,0.797,11,1234.000,19.281,30.000,0.000,6.273,0.091,0.143,6
3,1,12427,10,1,10,3.300,6636,4957,1679,1828.000,...,0.797,11,1234.000,19.281,30.000,0.000,6.273,0.909,0.909,1
4,1,13032,4,2,11,6.500,3867,2581,1286,1302.000,...,0.797,11,1234.000,19.281,30.000,0.000,6.273,0.364,0.400,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13307948,206209,43961,3,4,12,8.000,57037,36582,20455,22112.000,...,0.542,14,2472.000,20.600,30.000,3.000,9.500,0.214,0.273,2
13307949,206209,44325,1,7,7,8.000,3558,1471,2087,1278.000,...,0.542,14,2472.000,20.600,30.000,3.000,9.500,0.071,0.125,7
13307950,206209,48370,1,11,11,8.000,4070,2887,1183,1472.000,...,0.542,14,2472.000,20.600,30.000,3.000,9.500,0.071,0.250,3
13307951,206209,48697,1,7,7,6.000,9929,3645,6284,3416.000,...,0.542,14,2472.000,20.600,30.000,3.000,9.500,0.071,0.125,7


In [47]:
# orders_train_df = user_products[user_products.eval_set=='train']
# prior_train_df = user_products[['order_id','user_id','product_id','reordered','target']]

In [48]:
# prior_train_df.target.value_counts()
# prior_train_df

In [49]:
unique_user_product

,user_id,product_id,eval_set,target,order_id
0,112108,49302,train,1,1
1,112108,11109,train,1,1
2,112108,43633,train,1,1
3,112108,22035,train,1,1
4,79431,19660,train,1,36
...,...,...,...,...,...
8474656,117076,20539,prior,0,211941
8474657,117076,35221,prior,0,211941
8474658,117076,12861,prior,0,211941
8474659,175185,12023,prior,0,651501


In [50]:
full_df = unique_user_product.merge(user_product_full, how='left',on=['user_id','product_id'])
full_df

,user_id,product_id,eval_set,target,order_id,user_product_count,user_product_first_order,user_product_last_order,user_product_avg_basket_placement,product_ordered_vol,...,user_reorder_ratio,user_total_orders,user_lifetime_days,user_avg_days_between_orders,user_max_time_between_orders,user_min_time_between_orders,user_avg_cart_size,user_product_order_rate,user_product_reorder_rate,user_product_last_time_product_ordered
0,112108,49302,train,1,1,3,1,4,2.000,170,...,0.765,4,167.000,9.824,15.000,7.000,6.250,0.750,0.750,0
1,112108,11109,train,1,1,3,1,4,3.333,4579,...,0.765,4,167.000,9.824,15.000,7.000,6.250,0.750,0.750,0
2,112108,43633,train,1,1,3,1,4,4.000,663,...,0.765,4,167.000,9.824,15.000,7.000,6.250,0.750,0.750,0
3,112108,22035,train,1,1,3,1,4,6.000,61241,...,0.765,4,167.000,9.824,15.000,7.000,6.250,0.750,0.750,0
4,79431,19660,train,1,36,16,2,23,4.562,57921,...,0.500,23,3238.000,17.598,30.000,2.000,8.391,0.696,0.727,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8474656,117076,20539,prior,0,211941,1,1,1,5.000,29,...,0.216,10,1487.000,16.898,30.000,1.000,9.500,0.100,0.100,9
8474657,117076,35221,prior,0,211941,1,1,1,6.000,48022,...,0.216,10,1487.000,16.898,30.000,1.000,9.500,0.100,0.100,9
8474658,117076,12861,prior,0,211941,1,1,1,7.000,21,...,0.216,10,1487.000,16.898,30.000,1.000,9.500,0.100,0.100,9
8474659,175185,12023,prior,0,651501,1,23,23,6.000,3554,...,0.573,32,2242.000,9.877,30.000,1.000,7.719,0.031,0.100,9


In [51]:
# train_df.fillna(0, inplace=True)
# train_df.isnull().sum()

In [52]:
#checking for duplicates
full_df[full_df.duplicated(subset=['user_id','product_id','order_id'])==True]

,user_id,product_id,eval_set,target,order_id,user_product_count,user_product_first_order,user_product_last_order,user_product_avg_basket_placement,product_ordered_vol,...,user_reorder_ratio,user_total_orders,user_lifetime_days,user_avg_days_between_orders,user_max_time_between_orders,user_min_time_between_orders,user_avg_cart_size,user_product_order_rate,user_product_reorder_rate,user_product_last_time_product_ordered


In [53]:
del user_product_full

In [54]:
#get unique user-product where the user_id is in the train set 
# test  = full_df[full_df['target'] == 1 ]
# user_ids = test['user_id'].values
# full_df = full_df[full_df['user_id'].isin(user_ids)]
# full_df[full_df['user_id'].isin(user_ids)]

In [55]:
# train_df = full_df[full_df.target == 1]
# prior_df = full_df[full_df.target == 0]
# print(train_df.shape, prior_df.shape)

## Features Data Dictionary
- <b>user_id</b>:  <i>unique user </i>
- <b>product_id</b>:  <i>unique product per user  </i>
- <b>reordered</b>:  <i>Boolean, 1 if product was reordered by user </i>
***
#### Product Stats
`product_ordered_vol`:  <i>Total amount of this product ordered </i> \
`product_reordered_vol`:  <i>Number of times product reordered across all users </i> \
`product_ordered_once_vol`:  <i>Number of times product ordered once across all users </i> \
`product_ordered_twice_vol`:  <i>Number of times product ordered twice across all users </i> \
`product_order_twice_ratio`:  <i>product_ordered_twice_vol/product_ordered_once_vol </i> \
`product_avg_reorders`:  <i> product_reordered_vol / product_ordered_once_vol)+1</i> \
`product_overall_reorder_prob`:  <i> product_reordered_vol / product_ordered_vol </i> 
***
#### User Stats
`user_total_products`:  <i> Total number products ordered by user </i> \
`user_total_products_reordered`:  <i> Total number products reordered by user</i> \
`user_unique_product_count`:  <i>Number of unique products by user </i> \
`user_unique_reorder_count`:  <i> Number of unique reordered products by user</i> \
`user_unique_product_perc`:  <i> user_unique_product_count/user_total_products</i> \
`user_unique_reorder_perc`:  <i> user_unique_reorder_count/user_total_products_reordered</i> \
`user_total_items_after_first_order`:  <i> Number of products user ordered after first order</i> \
`user_reorder_ratio`:  <i>reorder_count/total_items_after_first_order </i> \
`user_total_orders`:  <i> Number of total orders by user</i> \
`user_lifetime_days`:  <i> Number of days each user has been active on Instacart</i> \
`user_avg_days_between_orders`:  <i>Average days between instacart orders </i> \
`user_max_time_between_orders`:  <i>Maximum days between instacart orders </i> \
`user_min_time_between_orders`:  <i>Minimum days between instacart orders </i> \
`user_avg_cart_size`:  <i>Average cart size by user </i> 
***
#### User-product Stats
`user_product_count`:  <i>Total times product was ordered by user </i> \
`user_product_first_order`:  <i>First order placed containing product by user </i> \
`user_product_last_order`:  <i>Last order placed containing product by user </i> \
`user_product_avg_basket_placement`:  <i>Average 'add_to_cart_order' for product by user </i> \
`user_product_order_rate`:  <i> user_product_count/user_total_orders</i> \
`user_product_reorder_rate`:  <i> user_product_count / (user_total_orders-user_product_first_order +1))</i> \
`user_product_last_time_product_ordered`:  <i>number of orders since user last ordered a product </i> 

## Save to CSV

In [56]:
full_df.to_csv('../../data/full_df.csv')
# train_df.to_csv('../../data/train_df.csv')
# prior_df.to_csv('../../data/prior_df.csv')
# user_products_test.to_csv('../../data/test_df.csv')

In [57]:
user_products.to_csv('../../data/user_products.csv')